In [45]:
import numpy as np
import pandas as pd
import pandashelpers
from sklearn.ensemble import RandomForestRegressor

In [46]:
metadata = pd.read_csv('metadata.csv').clean_column_names()

In [47]:
filament = metadata['filament_used'].str.strip(' ')\
                                    .str.split(' ',expand=True)\
                                    .rename(columns={0:'filament_used_mm',
                                                     1:'filament_used_cm3'
                                                    }
                                           )


In [48]:
filament['filament_used_mm'] = filament['filament_used_mm'].str.replace('mm','')\
                                                           .apply(float)

filament['filament_used_cm3'] = filament['filament_used_cm3'].str.strip('()')\
                                                             .str.replace('cm3','')\
                                                             .apply(float)

In [49]:
metadata = metadata.join(filament)

0        27.4
1        27.4
2        27.4
3        27.4
4        27.4
5        27.4
6        49.3
7        49.3
8        49.3
9       102.4
10      102.4
11      102.4
12      119.2
13      119.2
14      119.2
15       27.4
16       27.4
17       27.4
18       27.4
19       27.4
20       27.4
21       27.4
22       27.4
23       27.4
24       27.4
25       27.4
26       27.4
27       53.3
28       53.3
29       53.3
        ...  
1143     42.4
1144     42.4
1145     42.4
1146     45.8
1147     45.8
1148     45.8
1149     27.4
1150     27.4
1151     27.4
1152     27.4
1153     27.4
1154     27.4
1155     27.4
1156     27.4
1157     27.4
1158     27.4
1159     27.4
1160     27.4
1161     32.6
1162     32.6
1163     32.6
1164     41.4
1165     41.4
1166     41.4
1167     45.8
1168     45.8
1169     45.8
1170     27.4
1171     27.4
1172     27.4
Name: filament_used_cm3, Length: 1173, dtype: float64

In [63]:
y = metadata[metadata.columns.intersection(set(['filament_used_cm3']))]
x = metadata[metadata.columns.difference(set(['filament_used_cm3']))]
x_numeric = x.select_dtypes(include=np.number)
#x_non_numeric = x.select_dtypes(exclude=np.number)

In [64]:
#x = pd.get_dummies(x_non_numeric).join(x_numeric).dropna(axis=1)

In [65]:
regressor_rf = RandomForestRegressor()

In [67]:
regressor_rf.fit(x_numeric.dropna(1),y.values.reshape(-1))

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [71]:
regressor_rf.predict(x_numeric.dropna(1))

array([27.4, 27.4, 27.4, ..., 27.4, 27.4, 27.4])

In [68]:
pd.DataFrame(pd.Series(regressor_rf.feature_importances_,name='importance'),
             pd.Series(x.columns,name='feature')
            )

,importance
feature,
avoid_crossing_perimeters,NaN
bed_shape,NaN
bed_temperature,NaN
before_layer_gcode,NaN
bottom_solid_layers,NaN
bridge_acceleration,NaN
bridge_fan_speed,NaN
bridge_flow_ratio,NaN
bridge_speed,NaN


In [3]:
strength = pd.read_csv('data/3dprinter/data.csv')

In [4]:
controllable = ["print_speed",
                "layer_height",
                "infill_density",
                "nozzle_temperature",
                "wall_thickness"
                ]

input_args = ['--solid-infill-speed',
              '--layer-height',
              '--fill-density',
              '--temperature',
              '--solid-layers'
              ]


In [5]:
strength_summary = strength.deduplicate('print_speed',
                     numeric_agg='mean',
                     non_numeric_agg='first'
                    ).reset_index(drop=True)

In [51]:
metadata.merge(strength_summary,
               left_on=["--solid-infill-speed",
                        "--layer-height",
                        "--fill-density",
                        "--temperature"
                        ],
               right_on=["print_speed",
                         "layer_height",
                         "infill_density",
                         "nozzle_temperature"
                         ],
               how='outer').filter(regex='temp', axis='columns')


,--temperature,bed_temperature,first_layer_bed_temperature,first_layer_temperature,standby_temperature_delta,temperature,nozzle_temperature,bed_temperature
0,200.0,0.0,0.0,200.0,-5.0,200.0,NaN,NaN
1,200.0,0.0,0.0,200.0,-5.0,200.0,NaN,NaN
2,200.0,0.0,0.0,200.0,-5.0,200.0,NaN,NaN
3,200.0,0.0,0.0,200.0,-5.0,200.0,NaN,NaN
4,200.0,0.0,0.0,200.0,-5.0,200.0,NaN,NaN
5,200.0,0.0,0.0,200.0,-5.0,200.0,NaN,NaN
6,200.0,0.0,0.0,200.0,-5.0,200.0,NaN,NaN
7,200.0,0.0,0.0,200.0,-5.0,200.0,NaN,NaN
8,200.0,0.0,0.0,200.0,-5.0,200.0,NaN,NaN
9,200.0,0.0,0.0,200.0,-5.0,200.0,NaN,NaN


In [48]:
#metadata['--temperature']
#strength_summary['nozzle_temperature']

0     False
1     False
2      True
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13     True
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29     True
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38     True
39    False
40    False
41    False
42    False
43    False
44     True
45    False
46    False
47    False
48    False
49    False
Name: nozzle_temperature, dtype: bool